In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils import data

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from aif360.algorithms.postprocessing import (CalibratedEqOddsPostprocessing,
                                              EqOddsPostprocessing,
                                              RejectOptionClassification)
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
random_state = 1

mps


pip install 'aif360[AdversarialDebiasing]'


In [2]:
# read in data and split
X = torch.load('inputs/rfw_resnet50_face_embeddings.pt').cpu()
y = torch.load('inputs/rfw_resnet50_labels.pt').cpu()
df = pd.read_csv('inputs/rfw_resnet50_df.csv')
df['reference_ethnicity'] = df['reference_ethnicity'].str.lower()
ethnicity = df['reference_ethnicity']

In [3]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)

train_X = X[train_split]
X_val = X[val_split]
test_X = X[test_split]

train_y = y[train_split]
y_val = y[val_split]
test_y = y[test_split]

train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]


train_ethnicity = ethnicity[train_split].values
train_ethnicity[train_ethnicity=='caucasian'] = 0
train_ethnicity[train_ethnicity=='african'] = 1
train_ethnicity = train_ethnicity.astype(int)

ethnicity_val = ethnicity[val_split].values
ethnicity_val[ethnicity_val=='caucasian'] = 0
ethnicity_val[ethnicity_val=='african'] = 1
ethnicity_val = ethnicity_val.astype(int)

test_ethnicity = ethnicity[test_split].values
test_ethnicity[test_ethnicity=='caucasian'] = 0
test_ethnicity[test_ethnicity=='african'] = 1
test_ethnicity = test_ethnicity.astype(int)

In [4]:
train_matches = train_df[train_df.labels==1]
train_non_matches = train_df[train_df.labels==0]
print('non matches')
print(train_non_matches['reference_ethnicity'].value_counts())
print('matches')
print(train_matches['reference_ethnicity'].value_counts())

african_non_matches = train_non_matches[train_non_matches['reference_ethnicity']=='african']
caucasian_non_matches = train_non_matches[train_non_matches['reference_ethnicity']=='caucasian']

african_matches = train_matches[train_matches['reference_ethnicity']=='african']
caucasian_matches = train_matches[train_matches['reference_ethnicity']=='caucasian']

non matches
african      4492
caucasian    4376
Name: reference_ethnicity, dtype: int64
matches
african      4396
caucasian    4325
Name: reference_ethnicity, dtype: int64


In [5]:
np.random.seed(random_state)
african_matches_sub_idx = african_matches.index[np.random.choice(len(african_matches.index), size=4325, replace=False)]
caucasian_non_matches_sub_idx = caucasian_non_matches.index[np.random.choice(len(caucasian_non_matches.index), size=4325, replace=False)]
african_non_matches_sub_idx = african_non_matches.index[np.random.choice(len(african_non_matches.index), size=4325, replace=False)]
caucasian_matches_sub_idx = caucasian_matches.index

X_train = torch.cat([X[african_matches_sub_idx],X[caucasian_matches_sub_idx],X[african_non_matches_sub_idx],X[caucasian_non_matches_sub_idx]])
y_train = torch.cat([y[african_matches_sub_idx],y[caucasian_matches_sub_idx],y[african_non_matches_sub_idx],y[caucasian_non_matches_sub_idx]])
ethnicity_train = np.concatenate([ethnicity[african_matches_sub_idx],ethnicity[caucasian_matches_sub_idx],ethnicity[african_non_matches_sub_idx],ethnicity[caucasian_non_matches_sub_idx]])
df_train = pd.concat([df.iloc[african_matches_sub_idx],df.iloc[caucasian_matches_sub_idx],df.iloc[african_non_matches_sub_idx],df.iloc[caucasian_non_matches_sub_idx]])
df_train

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
6419,m.03j6s0,m.03j6s0,african,African,1.0
5541,m.01qd5bm,m.01qd5bm,african,African,1.0
5881,m.02qdp4v,m.02qdp4v,african,African,1.0
229,m.0272001,m.0272001,african,African,1.0
5264,m.05f962,m.05f962,african,African,1.0
...,...,...,...,...,...
18810,m.0g3f1p,m.01kxfvq,caucasian,Caucasian,0.0
20550,m.07gb0p,m.0bgwxt,caucasian,Caucasian,0.0
24018,m.07pyvy,m.01y7m1,caucasian,Caucasian,0.0
20879,m.02qg85,m.070p26,caucasian,Caucasian,0.0


In [6]:
# test_split, holdout_split = train_test_split(np.arange(5864),test_size=0.1, random_state=random_state)
X_test = test_X#[test_split]
# X_holdout = test_X[holdout_split]
y_test = test_y###[test_split]
# y_holdout = test_y##[holdout_split]
ethnicity_test = test_ethnicity#[test_split]
# ethnicity_holdout = test_ethnicity[holdout_split]
df_test = test_df#.iloc#[test_split]
# df_holdout = test_df.iloc[holdout_split]
df_test

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,labels
18904,m.0cx09_,m.02q_nsj,caucasian,Caucasian,0.0
28620,m.080gs5b,m.027w0nt,african,African,0.0
11687,m.027q3qg,m.027q3qg,caucasian,Caucasian,1.0
27477,m.034s_j,m.03h3058,african,African,0.0
12283,m.05p7_tb,m.05p7_tb,caucasian,Caucasian,1.0
...,...,...,...,...,...
18134,m.0hzpkcs,m.067fmb,caucasian,Caucasian,0.0
3639,m.05lnvt,m.05lnvt,african,African,1.0
11091,m.01ndxh,m.01ndxh,caucasian,Caucasian,1.0
9369,m.0c2g8y,m.0c2g8y,caucasian,Caucasian,1.0


In [7]:
val_matches = val_df[val_df.labels==1]
val_non_matches = val_df[val_df.labels==0]
print('non matches')
print(val_non_matches['reference_ethnicity'].value_counts())
print('matches')
print(val_matches['reference_ethnicity'].value_counts())

val_african_non_matches = val_non_matches[val_non_matches['reference_ethnicity']=='african']
val_african_non_matches = val_non_matches[val_non_matches['reference_ethnicity']=='african']

val_african_matches = val_matches[val_matches['reference_ethnicity']=='african']
val_african_matches = val_matches[val_matches['reference_ethnicity']=='african']

non matches
caucasian    1460
african      1454
Name: reference_ethnicity, dtype: int64
matches
caucasian    1480
african      1469
Name: reference_ethnicity, dtype: int64


In [8]:
test_matches = df_test[df_test.labels==1]
test_non_matches = df_test[df_test.labels==0]
print('non matches')
print(test_non_matches['reference_ethnicity'].value_counts())
print('matches')
print(test_matches['reference_ethnicity'].value_counts())

test_african_non_matches = test_non_matches[test_non_matches['reference_ethnicity']=='african']
test_african_non_matches = test_non_matches[test_non_matches['reference_ethnicity']=='african']

test_african_matches = test_matches[test_matches['reference_ethnicity']=='african']
test_african_matches = test_matches[test_matches['reference_ethnicity']=='african']

non matches
african      1518
caucasian    1358
Name: reference_ethnicity, dtype: int64
matches
african      1555
caucasian    1433
Name: reference_ethnicity, dtype: int64


In [9]:
# holdout_matches = df_holdout[df_holdout.labels==1]
# holdout_non_matches = df_holdout[df_holdout.labels==0]
# print('non matches')
# print(holdout_non_matches['reference_ethnicity'].value_counts())
# print('matches')
# print(holdout_matches['reference_ethnicity'].value_counts())

# holdout_african_non_matches = holdout_non_matches[holdout_non_matches['reference_ethnicity']=='african']
# holdout_african_non_matches = holdout_non_matches[holdout_non_matches['reference_ethnicity']=='african']

# holdout_african_matches = holdout_matches[holdout_matches['reference_ethnicity']=='african']
# holdout_african_matches = holdout_matches[holdout_matches['reference_ethnicity']=='african']

In [10]:
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)
def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
def AOE(tn_1,fp_1,fn_1,tp_1,tn_0,fp_0,fn_0,tp_0):
    tpr_1 = tp_1/(tp_1+fn_1)
    tpr_0 = tp_0/(tp_0+fn_0)

    fpr_1 = fp_1/(fp_1+tn_1)
    fpr_0 = fp_0/(fp_0+tn_0)


    return (np.abs(fpr_1-fpr_0) + np.abs(tpr_1 - tpr_0))/2

In [11]:
val_cos = cos_sim(X_val[:,:2048],X_val[:,2048:])
eer_thresh = 0.40895

eer_pred = (val_cos>eer_thresh)*1

df_bias = val_df.drop(['reference_identity','candidate_identity','candidate_ethnicity'],axis=1)
df_bias['eer_pred'] = eer_pred

df_bias_african = df_bias[df_bias['reference_ethnicity']=='african']
df_bias_caucasian = df_bias[df_bias['reference_ethnicity']=='caucasian']

african_eer_tn, african_eer_fp, african_eer_fn, african_eer_tp, african_eer_acc = confusion_mat(df_bias_african['eer_pred'], df_bias_african['labels'])
caucasian_eer_tn, caucasian_eer_fp, caucasian_eer_fn, caucasian_eer_tp, caucasian_eer_acc = confusion_mat(df_bias_caucasian['eer_pred'], df_bias_caucasian['labels'])

african_eer_tnr = african_eer_tn/(african_eer_tn+african_eer_fp)
african_eer_tpr = african_eer_tp/(african_eer_tp+african_eer_fn)
african_eer_fnr = african_eer_fn/(african_eer_fn+african_eer_tp)
african_eer_fpr = african_eer_fp/(african_eer_tn+african_eer_fp)

caucasian_eer_tnr = caucasian_eer_tn/(caucasian_eer_tn+caucasian_eer_fp)
caucasian_eer_tpr = caucasian_eer_tp/(caucasian_eer_tp+caucasian_eer_fn)
caucasian_eer_fnr = caucasian_eer_fn/(caucasian_eer_fn+caucasian_eer_tp)
caucasian_eer_fpr = caucasian_eer_fp/(caucasian_eer_tn+caucasian_eer_fp)

print(african_eer_fnr/caucasian_eer_fnr)
print(african_eer_fpr/caucasian_eer_fpr)
print(caucasian_eer_tnr/african_eer_tnr)
print(caucasian_eer_tpr/african_eer_tpr)
print((1-african_eer_acc)/(1-caucasian_eer_acc))
print('african fnr:',african_eer_fnr,'caucasian fnr:',caucasian_eer_fnr)
print('african fpr:',african_eer_fpr,'caucasian fpr:',caucasian_eer_fpr)
print('caucasian tnr:',caucasian_eer_tnr,'african tnr:',african_eer_tnr)
print('caucasian tpr:',caucasian_eer_tpr,'african tpr:',african_eer_tpr)
print('african mis class:',(1-african_eer_acc),'caucasian mis class:',(1-caucasian_eer_acc))
print('african  class:',(african_eer_acc),'caucasian  class:',(caucasian_eer_acc))

0.4066924388431249
14.950328595445516
1.09472498962225
0.9549544807965862
1.5172477777069073
african fnr: 0.029952348536419333 caucasian fnr: 0.07364864864864865
african fpr: 0.0921595598349381 caucasian fpr: 0.0061643835616438354
caucasian tnr: 0.9938356164383562 african tnr: 0.9078404401650619
caucasian tpr: 0.9263513513513514 african tpr: 0.9700476514635806
african mis class: 0.06089633937735206 caucasian mis class: 0.04013605442176871
african  class: 0.9391036606226479 caucasian  class: 0.9598639455782313


In [12]:
print(african_eer_tn, african_eer_fp, african_eer_fn, african_eer_tp)
print(caucasian_eer_tn, caucasian_eer_fp, caucasian_eer_fn, caucasian_eer_tp)

1320 134 44 1425
1451 9 109 1371


In [13]:
ethnicity_val[ethnicity_val=='caucasian'] = 0
ethnicity_val[ethnicity_val=='african'] = 1
ethnicity_val = ethnicity_val.astype(int)
ethnicity_val
val_bias_df = pd.DataFrame(columns=['label','prediction','protected','rounded'])
val_bias_df['label']=y_val.cpu().numpy()
val_bias_df['prediction']=val_cos.cpu().numpy()
val_bias_df['rounded']=eer_pred.cpu().numpy()
val_bias_df['protected']=ethnicity_val
val_bias_df
unpriveleged_df = val_bias_df[val_bias_df['protected']==1]
unpriveleged_df
priveleged_df = val_bias_df[val_bias_df['protected']==0]
priveleged_df

,label,prediction,protected,rounded
0,1.0,0.557137,0,1
2,0.0,0.203042,0,0
7,0.0,0.216037,0,0
9,0.0,0.235024,0,0
12,0.0,0.212539,0,0
...,...,...,...,...
5857,1.0,0.819825,0,1
5858,1.0,0.738980,0,1
5859,1.0,0.561938,0,1
5860,0.0,0.279173,0,0


In [14]:
confusion_mat(val_bias_df['rounded'], val_bias_df['label'])

(2771, 143, 153, 2796, 0.9495139007334129)

In [15]:
unpriveleged_tn, unpriveleged_fp, unpriveleged_fn, unpriveleged_tp, unpriveleged_acc = confusion_mat(unpriveleged_df['rounded'], unpriveleged_df['label'])
priveleged_tn, priveleged_fp, priveleged_fn, priveleged_tp, priveleged_acc = confusion_mat(priveleged_df['rounded'], priveleged_df['label'])
unpriveleged_fnr = unpriveleged_fn/(unpriveleged_fn+unpriveleged_tp)
unpriveleged_fpr = unpriveleged_fp/(unpriveleged_fp+unpriveleged_tn)
priveleged_fnr = priveleged_fn/(priveleged_fn+priveleged_tp)
priveleged_fpr = priveleged_fp/(priveleged_fp+priveleged_tn)

print('Overall Accuracy:',round(torch.sum(eer_pred == y_val).item()/y_val.size()[0],3))
print('Missclassificaition Rate Ratio:',round((1-unpriveleged_acc)/(1- priveleged_acc),3))
print('FNR Ratio:',round(unpriveleged_fnr/priveleged_fnr,3),'FPR Ratio:',round(unpriveleged_fpr/priveleged_fpr,3))
print('AOE:',round(AOE(unpriveleged_tn,unpriveleged_fp,unpriveleged_fn,unpriveleged_tp,priveleged_tn,priveleged_fp,priveleged_fn,priveleged_tp),3))

Overall Accuracy: 0.95
Missclassificaition Rate Ratio: 1.517
FNR Ratio: 0.407 FPR Ratio: 14.95
AOE: 0.065


In [16]:
def to_dataframe(y_true, y_pred, y_prot):
        y_true, y_pred, y_prot = y_true.float().cpu().numpy(), y_pred.float().cpu().numpy(), y_prot.astype(np.float32)
        df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'y_prot': y_prot})
        dataset = StandardDataset(df, 'y_true', [1.], ['y_prot'], [[1.]])
        dataset.scores = y_pred.reshape(-1, 1)
        return dataset

def compute_bias(y_pred, y_true, prot, metric):
    """Compute bias on the dataset"""
    def zero_if_nan(data):
        """Zero if there is a nan"""
        return 0. if torch.isnan(data) else data

    gtpr_prot = zero_if_nan(y_pred[prot * y_true == 1].mean())
    gfpr_prot = zero_if_nan(y_pred[prot * (1-y_true) == 1].mean())
    mean_prot = zero_if_nan(y_pred[prot == 1].mean())

    gtpr_unprot = zero_if_nan(y_pred[(1-prot) * y_true == 1].mean())
    gfpr_unprot = zero_if_nan(y_pred[(1-prot) * (1-y_true) == 1].mean())
    mean_unprot = zero_if_nan(y_pred[(1-prot) == 1].mean())

    if metric == "spd":
        return mean_prot - mean_unprot
    elif metric == "aod":
        return 0.5 * ((gfpr_prot - gfpr_unprot) + (gtpr_prot - gtpr_unprot))
    elif metric == "eod":
        return gtpr_prot - gtpr_unprot
    elif metric=='false_diff':
        return (np.abs(gfpr_prot - gfpr_unprot) + np.abs(gtpr_unprot - gtpr_prot))

In [17]:
aif_data = to_dataframe(y_val, val_cos, ethnicity_val)


In [18]:
eo = EqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                  unprivileged_groups=[{'y_prot': 1.}],seed=random_state)

eo=eo.fit(aif_data,aif_data)

eo_pred = eo.predict(aif_data).labels.reshape(-1)
eo_label = aif_data.labels.reshape(-1)
eo_prot = aif_data.protected_attributes.reshape(-1)

eo_unpriv_ids = (eo_prot==1)
eo_unpriv_pred = eo_pred[eo_unpriv_ids]
eo_unpriv_label = eo_label[eo_unpriv_ids]

eo_priv_ids = (eo_prot==0)
eo_priv_pred = eo_pred[eo_priv_ids]
eo_priv_label = eo_label[eo_priv_ids]

eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp, eo_unpriv_acc = confusion_mat(eo_unpriv_pred, eo_unpriv_label)
eo_unpriv_fpr = eo_unpriv_fp/(eo_unpriv_fp+eo_unpriv_tn)
eo_unpriv_fnr = eo_unpriv_fn/(eo_unpriv_fn+eo_unpriv_tp)

eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp, eo_priv_acc = confusion_mat(eo_priv_pred, eo_priv_label)
eo_priv_fpr = eo_priv_fp/(eo_priv_fp+eo_priv_tn)
eo_priv_fnr = eo_priv_fn/(eo_priv_fn+eo_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-eo_unpriv_acc)/(1- eo_priv_acc),3))
print('FPR Ratio:',eo_unpriv_fpr/eo_priv_fpr)
print('FNR Ratio:',eo_unpriv_fnr/eo_priv_fnr)
print(np.sum(eo_pred == eo_label)/len(eo_pred))
print('AOE:',round(AOE(eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp,eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp),3))


Missclassificaition Rate Ratio: nan
FPR Ratio: nan
FNR Ratio: nan
1.0
AOE: 0.0


In [19]:
ethnicity_test[ethnicity_test=='caucasian'] = 0
ethnicity_test[ethnicity_test=='african'] = 1
ethnicity_test = ethnicity_test.astype(int)
ethnicity_test
test_bias_df = pd.DataFrame(columns=['label','prediction','protected','rounded'])
test_bias_df['label']=y_test.cpu().numpy()
test_cos = cos_sim(X_test[:,:2048],X_test[:,2048:])
test_eer_pred = (test_cos>eer_thresh)*1
test_bias_df['prediction']=test_cos.cpu().numpy()
test_bias_df['rounded']=test_eer_pred.cpu().numpy()
test_bias_df['protected']=ethnicity_test

test_unpriveleged_df = test_bias_df[test_bias_df['protected']==1]

test_priveleged_df = test_bias_df[test_bias_df['protected']==0]

test_aif_data = to_dataframe(y_test, test_cos, ethnicity_test)

test_eo_pred = eo.predict(test_aif_data).labels.reshape(-1)
test_eo_label = test_aif_data.labels.reshape(-1)
test_eo_prot = test_aif_data.protected_attributes.reshape(-1)

test_eo_unpriv_ids = (test_eo_prot==1)
test_eo_unpriv_pred = test_eo_pred[test_eo_unpriv_ids]
test_eo_unpriv_label = test_eo_label[test_eo_unpriv_ids]

test_eo_priv_ids = (test_eo_prot==0)
test_eo_priv_pred = test_eo_pred[test_eo_priv_ids]
test_eo_priv_label = test_eo_label[test_eo_priv_ids]

test_eo_unpriv_tn, test_eo_unpriv_fp, test_eo_unpriv_fn, test_eo_unpriv_tp, test_eo_unpriv_acc = confusion_mat(test_eo_unpriv_pred, test_eo_unpriv_label)
test_eo_unpriv_fpr = test_eo_unpriv_fp/(test_eo_unpriv_fp+test_eo_unpriv_tn)
test_eo_unpriv_fnr = test_eo_unpriv_fn/(test_eo_unpriv_fn+test_eo_unpriv_tp)

test_eo_priv_tn, test_eo_priv_fp, test_eo_priv_fn, test_eo_priv_tp, test_eo_priv_acc = confusion_mat(test_eo_priv_pred, test_eo_priv_label)
test_eo_priv_fpr = test_eo_priv_fp/(test_eo_priv_fp+test_eo_priv_tn)
test_eo_priv_fnr = test_eo_priv_fn/(test_eo_priv_fn+test_eo_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-test_eo_unpriv_acc)/(1- test_eo_priv_acc),3))
print('FPR Ratio:',test_eo_unpriv_fpr/test_eo_priv_fpr)
print('FNR Ratio:',test_eo_unpriv_fnr/test_eo_priv_fnr)
print(np.sum(test_eo_pred == test_eo_label)/len(test_eo_pred))
print('AOE:',round(AOE(test_eo_unpriv_tn, test_eo_unpriv_fp, test_eo_unpriv_fn, test_eo_unpriv_tp,test_eo_priv_tn, test_eo_priv_fp, test_eo_priv_fn, test_eo_priv_tp),3))

Missclassificaition Rate Ratio: nan
FPR Ratio: nan
FNR Ratio: nan
1.0
AOE: 0.0


In [20]:
print('tnm', 'fm', 'fnm', 'tm')
print(confusion_mat(test_eo_unpriv_pred, test_eo_unpriv_label))
print(confusion_mat(test_eo_priv_pred, test_eo_priv_label))
print(test_eo_unpriv_fpr,test_eo_unpriv_fnr)
print(test_eo_priv_fpr,test_eo_priv_fnr)
confusion_mat(test_eo_pred, test_eo_label)

tnm fm fnm tm
(1518, 0, 0, 1555, 1.0)
(1358, 0, 0, 1433, 1.0)
0.0 0.0
0.0 0.0


(2876, 0, 0, 2988, 1.0)

In [21]:
cost_constraint = 'weighted'
privileged_groups=[{'y_prot': 0.}],
unprivileged_groups=[{'y_prot': 1.}]
cpp = CalibratedEqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                             unprivileged_groups=[{'y_prot': 1.}],
                                             cost_constraint=cost_constraint,seed=random_state)
cpp=cpp.fit(aif_data,aif_data)
cpp_pred = cpp.predict(aif_data).labels.reshape(-1)
cpp_label = aif_data.labels.reshape(-1)
cpp_prot = aif_data.protected_attributes.reshape(-1)

unpriv_ids = (cpp_prot==1)
unpriv_pred = cpp_pred[unpriv_ids]
unpriv_label = cpp_label[unpriv_ids]

priv_ids = (cpp_prot==0)
priv_pred = cpp_pred[priv_ids]
priv_label = cpp_label[priv_ids]

unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp, unpriv_acc = confusion_mat(unpriv_pred, unpriv_label)
unpriv_fpr = unpriv_fp/(unpriv_fp+unpriv_tn)
unpriv_fnr = unpriv_fn/(unpriv_fn+unpriv_tp)

priv_tn, priv_fp, priv_fn, priv_tp, priv_acc = confusion_mat(priv_pred, priv_label)
priv_fpr = priv_fp/(priv_fp+priv_tn)
priv_fnr = priv_fn/(priv_fn+priv_tp)

print('Missclassificaition Rate Ratio:',round((1-unpriv_acc)/(1- priv_acc),3))
print('FPR Ratio:',round(unpriv_fpr/priv_fpr,3))
print('FNR Ratio:',round(unpriv_fnr/priv_fnr,3))
print('Overall Accuracy:',round(np.sum(cpp_pred == cpp_label)/len(cpp_label),3))
print('AOE:',round(AOE(unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp,priv_tn, priv_fp, priv_fn, priv_tp),3))

Missclassificaition Rate Ratio: 0.548
FPR Ratio: 0.123
FNR Ratio: 0.761
Overall Accuracy: 0.871
AOE: 0.075


In [22]:
test_cpp_pred = cpp.predict(test_aif_data).labels.reshape(-1)
test_cpp_label = test_aif_data.labels.reshape(-1)
test_cpp_prot = test_aif_data.protected_attributes.reshape(-1)

test_cpp_unpriv_ids = (test_cpp_prot==1)
test_cpp_unpriv_pred = test_cpp_pred[test_cpp_unpriv_ids]
test_cpp_unpriv_label = test_cpp_label[test_cpp_unpriv_ids]

test_cpp_priv_ids = (test_cpp_prot==0)
test_cpp_priv_pred = test_cpp_pred[test_cpp_priv_ids]
test_cpp_priv_label = test_cpp_label[test_cpp_priv_ids]

test_cpp_unpriv_tn, test_cpp_unpriv_fp, test_cpp_unpriv_fn, test_cpp_unpriv_tp, test_cpp_unpriv_acc = confusion_mat(test_cpp_unpriv_pred, test_cpp_unpriv_label)
test_cpp_unpriv_fpr = test_cpp_unpriv_fp/(test_cpp_unpriv_fp+test_cpp_unpriv_tn)
test_cpp_unpriv_fnr = test_cpp_unpriv_fn/(test_cpp_unpriv_fn+test_cpp_unpriv_tp)

test_cpp_priv_tn, test_cpp_priv_fp, test_cpp_priv_fn, test_cpp_priv_tp, test_cpp_priv_acc = confusion_mat(test_cpp_priv_pred, test_cpp_priv_label)
test_cpp_priv_fpr = test_cpp_priv_fp/(test_cpp_priv_fp+test_cpp_priv_tn)
test_cpp_priv_fnr = test_cpp_priv_fn/(test_cpp_priv_fn+test_cpp_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-test_cpp_unpriv_acc)/(1- test_cpp_priv_acc),3))
print('FPR Ratio:',test_cpp_unpriv_fpr/test_cpp_priv_fpr)
print('FNR Ratio:',test_cpp_unpriv_fnr/test_cpp_priv_fnr)
print(np.sum(test_cpp_pred == test_cpp_label)/len(test_cpp_pred))
print('AOD:',compute_bias(torch.tensor(test_cpp_pred), torch.tensor(test_cpp_label), torch.tensor(test_cpp_prot), 'aod'))
print('False Diff:',compute_bias(torch.tensor(test_cpp_pred), torch.tensor(test_cpp_label), torch.tensor(test_cpp_prot), 'false_diff'))

Missclassificaition Rate Ratio: 0.547
FPR Ratio: 0.09387758421574846
FNR Ratio: 0.81070485409001
0.876193724420191
AOD: tensor(-0.0348, dtype=torch.float64)
False Diff: tensor(0.1465, dtype=torch.float64)


In [23]:
print('tnm', 'fm', 'fnm', 'tm')
print(confusion_mat(test_cpp_unpriv_pred, test_cpp_unpriv_label))
print(confusion_mat(test_cpp_priv_pred, test_cpp_priv_label))
print(test_cpp_unpriv_fpr,test_cpp_unpriv_fnr)
print(test_cpp_priv_fpr,test_cpp_priv_fnr)
confusion_mat(test_cpp_pred, test_cpp_label)

tnm fm fnm tm
(1501, 17, 256, 1299, 0.9111617312072893)
(1196, 162, 291, 1142, 0.8376925833034754)
0.011198945981554678 0.16463022508038586
0.11929307805596466 0.2030704815073273


(2697, 179, 547, 2441, 0.876193724420191)

In [24]:
roc = RejectOptionClassification(unprivileged_groups=[{'y_prot': 1.}],
                                         privileged_groups=[{'y_prot': 0.}],
                                         low_class_thresh=0.01, high_class_thresh=0.99,
                                         num_class_thresh=100, num_ROC_margin=50,
                                         metric_name="Average odds difference",
                                         metric_ub=0.05, metric_lb=-0.05)

roc = roc.fit(aif_data, aif_data)

roc_pred = roc.predict(aif_data).labels.reshape(-1)
roc_label = aif_data.labels.reshape(-1)
roc_prot = aif_data.protected_attributes.reshape(-1)

roc_unpriv_ids = (roc_prot==1)
roc_unpriv_pred = roc_pred[roc_unpriv_ids]
roc_unpriv_label = roc_label[roc_unpriv_ids]

roc_priv_ids = (roc_prot==0)
roc_priv_pred = roc_pred[roc_priv_ids]
roc_priv_label = roc_label[roc_priv_ids]

roc_unpriv_tn, roc_unpriv_fp, roc_unpriv_fn, roc_unpriv_tp, roc_unpriv_acc = confusion_mat(roc_unpriv_pred, roc_unpriv_label)
roc_unpriv_fpr = roc_unpriv_fp/(roc_unpriv_fp+roc_unpriv_tn)
roc_unpriv_fnr = roc_unpriv_fn/(roc_unpriv_fn+roc_unpriv_tp)

roc_priv_tn, roc_priv_fp, roc_priv_fn, roc_priv_tp, roc_priv_acc = confusion_mat(roc_priv_pred, roc_priv_label)
roc_priv_fpr = roc_priv_fp/(roc_priv_fp+roc_priv_tn)
roc_priv_fnr = roc_priv_fn/(roc_priv_fn+roc_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-roc_unpriv_acc)/(1- roc_priv_acc),3))
print('FPR Ratio:',round(roc_unpriv_fpr/roc_priv_fpr,3))
print('FNR Ratio:',round(roc_unpriv_fnr/roc_priv_fnr,3))
print(np.sum(roc_pred == roc_label)/len(roc_pred))
print('AOE:',round(AOE(roc_unpriv_tn, roc_unpriv_fp, roc_unpriv_fn, roc_unpriv_tp,roc_priv_tn, roc_priv_fp, roc_priv_fn, roc_priv_tp),3))


Missclassificaition Rate Ratio: 0.948
FPR Ratio: 13.455
FNR Ratio: 0.537
0.9479788504178748
AOE: 0.045


In [25]:
test_roc_pred = roc.predict(test_aif_data).labels.reshape(-1)
test_roc_label = test_aif_data.labels.reshape(-1)
test_roc_prot = test_aif_data.protected_attributes.reshape(-1)

test_roc_unpriv_ids = (test_roc_prot==1)
test_roc_unpriv_pred = test_roc_pred[test_roc_unpriv_ids]
test_roc_unpriv_label = test_roc_label[test_roc_unpriv_ids]

test_roc_priv_ids = (test_roc_prot==0)
test_roc_priv_pred = test_roc_pred[test_roc_priv_ids]
test_roc_priv_label = test_roc_label[test_roc_priv_ids]

test_roc_unpriv_tn, test_roc_unpriv_fp, test_roc_unpriv_fn, test_roc_unpriv_tp, test_roc_unpriv_acc = confusion_mat(test_roc_unpriv_pred, test_roc_unpriv_label)
test_roc_unpriv_fpr = test_roc_unpriv_fp/(test_roc_unpriv_fp+test_roc_unpriv_tn)
test_roc_unpriv_fnr = test_roc_unpriv_fn/(test_roc_unpriv_fn+test_roc_unpriv_tp)

test_roc_priv_tn, test_roc_priv_fp, test_roc_priv_fn, test_roc_priv_tp, test_roc_priv_acc = confusion_mat(test_roc_priv_pred, test_roc_priv_label)
test_roc_priv_fpr = test_roc_priv_fp/(test_roc_priv_fp+test_roc_priv_tn)
test_roc_priv_fnr = test_roc_priv_fn/(test_roc_priv_fn+test_roc_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-test_roc_unpriv_acc)/(1- test_roc_priv_acc),3))
print('FPR Ratio:',test_roc_unpriv_fpr/test_roc_priv_fpr)
print('FNR Ratio:',test_roc_unpriv_fnr/test_roc_priv_fnr)
print(np.sum(test_roc_pred == test_roc_label)/len(test_roc_pred))
print('AOD:',compute_bias(torch.tensor(test_roc_pred), torch.tensor(test_roc_label), torch.tensor(test_roc_prot), 'aod'))
print('False Diff:',compute_bias(torch.tensor(test_roc_pred), torch.tensor(test_roc_label), torch.tensor(test_roc_prot), 'false_diff'))

Missclassificaition Rate Ratio: 0.882
FPR Ratio: 5.4314888010540185
FNR Ratio: 0.486529128637335
0.9411664392905866
AOD: tensor(0.0517, dtype=torch.float64)
False Diff: tensor(0.1034, dtype=torch.float64)


In [26]:
print('tnm', 'fm', 'fnm', 'tm')
print(confusion_mat(test_roc_unpriv_pred, test_roc_unpriv_label))
print(confusion_mat(test_roc_priv_pred, test_roc_priv_label))
print(test_roc_unpriv_fpr,test_roc_unpriv_fnr)
print(test_roc_priv_fpr,test_roc_priv_fnr)
confusion_mat(test_roc_pred, test_roc_label)

tnm fm fnm tm
(1433, 85, 85, 1470, 0.9446794663195575)
(1344, 14, 161, 1272, 0.9372984593335721)
0.055994729907773384 0.05466237942122187
0.010309278350515464 0.11235170969993022


(2777, 99, 246, 2742, 0.9411664392905866)